![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Tokenización de textos  

En este notebook aprenderá a tokenizar un texto usando la librería especializada sklearn y [nltk](https://www.nltk.org/).

Este notebook tiene la licencia de [Creative Commons Attribution-ShareAlike 3.0 Unported License](http://creativecommons.org/licenses/by-sa/3.0/deed.en_US). Un agradecimiento especial para [Kevin Markham](https://github.com/justmarkham)

## Instrucciones Generales:

La tokenización es un proceso primordial para la limpieza de datos de texto que permite mejorar el performance de los modelos predictivos de procesamiento de lenguaje natural. Por medio de este notebook deberá tokenizar el texto del set de noticias populares de UCL. Para conocer más detalles de la base puede ingresar al siguiente [vínculo](https://archive.ics.uci.edu/ml/datasets/online+news+popularity#).
   
Para realizar la actividad, solo siga las indicaciones asociadas a cada celda del notebook.

### Importar base de datos y librerías

In [1]:
# SUGERIDO: Descomenta la siguiente linea de código si requieres instalar las libreías básicas utilizadas en este notebook
# Si requieres incluir más librerías puedes agregarlas al archivo Semana 4\requirements.txt
#!pip install -r requirements.txt

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Importación librerías
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

%matplotlib inline

In [4]:
# Carga de datos de archivos .csv
df = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2025/main/datasets/mashable_texts.csv', index_col=0)
df.head()

,author,author_web,shares,text,title,facebo,google,linked,twitte,twitter_followers
0,Seth Fiegerman,http://mashable.com/people/seth-fiegerman/,4900,\nApple's long and controversial ebook case ha...,The Supreme Court smacked down Apple today,http://www.facebook.com/sfiegerman,NaN,http://www.linkedin.com/in/sfiegerman,https://twitter.com/sfiegerman,14300
1,Rebecca Ruiz,http://mashable.com/people/rebecca-ruiz/,1900,Analysis\n\n\n\n\n\nThere is a reason that Don...,Every woman has met a man like Donald Trump,NaN,NaN,NaN,https://twitter.com/rebecca_ruiz,3738
2,Davina Merchant,http://mashable.com/people/568bdab351984019310...,7000,LONDON - Last month we reported on a dog-sized...,Adorable dog-sized rabbit finally finds his fo...,NaN,https://plus.google.com/105525238342980116477?...,NaN,NaN,0
3,Scott Gerber,[],5000,Today's digital marketing experts must have a ...,15 essential skills all digital marketing hire...,NaN,NaN,NaN,NaN,0
4,Josh Dickey,http://mashable.com/people/joshdickey/,1600,"LOS ANGELES — For big, fun, populist popcorn m...",Mashable top 10: 'The Force Awakens' is the be...,NaN,https://plus.google.com/109213469090692520544?...,NaN,https://twitter.com/JLDlite,11200


### Crear varaible de interés

In [5]:
# Separación de variable de interés (y)
y = df.shares
y.describe()

,shares
count,82.000000
mean,3090.487805
std,8782.031594
min,437.000000
25%,893.500000
50%,1200.000000
75%,2275.000000
max,63100.000000


In [6]:
# Categorización de la variable de interés (y)
y = pd.cut(y, [0, 893, 1200, 2275, 63200], labels=[0, 1, 2, 3])
y.value_counts()

,count
shares,
1,22
0,21
3,21
2,18


In [7]:
# Definición de variable de interés en el dataframe
df['y'] = y

### Crear variables predictoras X_A - tokenización sin limpieza

In [8]:
# Separación de variables predictoras (X), solo se considera el texto de la noticia
X = df.text

In [9]:
# Creación de matrices de documentos usando CountVectorizer a partir de X
vect_A = CountVectorizer()
X_dtm_A = vect_A.fit_transform(X)
temp_A=X_dtm_A.todense()

In [10]:
# Visualización de diccionario de palabras con su respectivo ID asignado
vect_A.vocabulary_

{'apple': 682,
 'long': 4303,
 'and': 617,
 'controversial': 1747,
 'ebook': 2401,
 'case': 1307,
 'has': 3367,
 'reached': 5734,
 'its': 3884,
 'final': 2893,
 'chapter': 1383,
 'it': 3878,
 'not': 4883,
 'the': 7054,
 'happy': 3352,
 'ending': 2527,
 'company': 1612,
 'wanted': 7620,
 'supreme': 6865,
 'court': 1809,
 'on': 4969,
 'monday': 4687,
 'rejected': 5841,
 'an': 603,
 'appeal': 673,
 'filed': 2882,
 'by': 1224,
 'to': 7150,
 'overturn': 5075,
 'stinging': 6723,
 'ruling': 6087,
 'that': 7051,
 'led': 4181,
 'broad': 1147,
 'conspiracy': 1706,
 'with': 7748,
 'several': 6303,
 'major': 4374,
 'publishers': 5610,
 'fix': 2927,
 'price': 5483,
 'of': 4935,
 'books': 1088,
 'sold': 6528,
 'through': 7106,
 'online': 4979,
 'bookstore': 1089,
 'decision': 2009,
 'means': 4496,
 'now': 4895,
 'no': 4858,
 'choice': 1437,
 'but': 1215,
 'pay': 5178,
 'out': 5037,
 '400': 223,
 'million': 4611,
 'consumers': 1714,
 'additional': 446,
 '50': 252,
 'in': 3664,
 'legal': 4187,
 'fees'

In [11]:
# Impresión de dimensiones de matriz de documentos donde las filas son documentos y las columnas son términos o tokens
X_dtm_A.shape

(82, 7969)

In [12]:
# Visualización de 50 términos en el diccionario de palabras
print(vect_A.get_feature_names_out()[-150:-100])

['ydwnm50jlu' 'ye' 'yeah' 'year' 'years' 'yec' 'yeezy' 'yellow' 'yelp'
 'yep' 'yes' 'yesterday' 'yesweather' 'yet' 'yoga' 'yong' 'york' 'you'
 'young' 'younger' 'youngest' 'your' 'yourself' 'youth' 'youtube'
 'youtubeduck' 'yup' 'yuyuan' 'yücel' 'zach' 'zaxoqbv487' 'zero'
 'zgkymde1lzewlza0l2zkl1n0yxj0dxayljq0mdvhlmpwzwpwcxrodw1ictexnxgxmtujcmujanbn'
 'zgkymde1lzewlza0l2zkl1n0yxj0dxayljq0mdvhlmpwzwpwcxrodw1icteymdb4nji3iwplcwpwzw'
 'zgkymde1lzewlza0l2zkl1n0yxj0dxayljq0mdvhlmpwzwpwcxrodw1icti4ohgxnjijcmujanbn'
 'zgkymde1lzewlza0l2zkl1n0yxj0dxayljq0mdvhlmpwzwpwcxrodw1ictk1mhg1mzqjcmujanbn'
 'zgkymde1lzewlza0l2zkl1n0yxj0dxayljq0mdvhlmpwzwpwcxrodw1ictu2mhg3ntakzqlqcgc'
 'zgkymde1lzewlza0l2zkl1n0yxj0dxayljq0mdvhlmpwzwpwcxrodw1ictywmhgzmzgjcmujanbn'
 'zgkymde1lzewlza0lzm1l2jpcmrfdgfudhj1lmu3zwmzlmpwzwpwcxrodw1ictexnxgxmtujcmujanbn'
 'zgkymde1lzewlza0lzm1l2jpcmrfdgfudhj1lmu3zwmzlmpwzwpwcxrodw1icteymdb4nji3iwplcwpwzw'
 'zgkymde1lzewlza0lzm1l2jpcmrfdgfudhj1lmu3zwmzlmpwzwpwcxrodw1icti4ohgxnjijcm

### Crear variables predictoras X_B - tokenización con limpieza de mayúsculas

In [13]:
# Creación de matrices de documentos usando CountVectorizer a partir de X, volviendo todas la palabras en minúscula
# a partir del parámetro 'lowercase=False'
vect_B = CountVectorizer(lowercase=False)
X_dtm_B = vect_B.fit_transform(X)

In [14]:
# Impresión dimensiones de matriz de documentos donde las filas son documentos y las columnas son términos o tokens
X_dtm_B.shape

(82, 8759)

In [15]:
# Visualización de 50 términos en el diccionario de palabras
print(vect_B.get_feature_names_out()[-150:-100])

['weighed' 'weird' 'welcome' 'welcomed' 'welcoming' 'welfare' 'well'
 'wells' 'went' 'were' 'weren' 'west' 'western' 'what' 'whatever'
 'whatsoever' 'wheel' 'wheelchair' 'wheeliz' 'wheels' 'when' 'where'
 'wherever' 'whether' 'which' 'while' 'whistles' 'white' 'who' 'whole'
 'wholesome' 'whose' 'why' 'wide' 'widely' 'wider' 'widespread' 'widgets'
 'width' 'wife' 'wildest' 'wildly' 'will' 'willing' 'willrahn' 'win'
 'wind' 'window' 'windows' 'windscreen']


### Crear variables predictoras X_C - tokenización con limpieza de mayúsculas y usando n-gramas

In [16]:
# Creación de matrices de documentos usando CountVectorizer a partir de X y usando n-gramas
# a partir del parámetro 'ngram_range=(1, 4)'
vect_C = CountVectorizer(lowercase=False, ngram_range=(1, 4))
X_dtm_C = vect_C.fit_transform(X)

In [17]:
# Impresión de dimensiones de matriz de documentos, donde las filas son documentos y las columnas son términos o tokens
X_dtm_C.shape

(82, 116956)

In [18]:
# Visualización de 50 términos en el diccionario de palabras
print(vect_C.get_feature_names_out()[-150:-100])

['your scope of knowledge' 'your score' 'your score the'
 'your score the more' 'your skills' 'your skills in'
 'your skills in variety' 'your skin' 'your skin The'
 'your skin The fabric' 'your smart' 'your smart phone'
 'your smart phone runs' 'your smartphone' 'your smartphone and'
 'your smartphone and go' 'your software' 'your software and'
 'your software and that' 'your sorrows' 'your sorrows in'
 'your sorrows in bag' 'your specific' 'your specific company'
 'your specific company For' 'your startup' 'your startup should'
 'your startup should look' 'your tablet' 'your tablet and'
 'your tablet and start' 'your tasty' 'your tasty souvenirs'
 'your tasty souvenirs South' 'your three' 'your three main'
 'your three main movement' 'your time' 'your time and'
 'your time and should' 'your time to' 'your time to The' 'your toddler'
 'your toddler doesn' 'your toddler doesn have' 'your toolbox'
 'your toolbox If' 'your toolbox If you' 'your tush' 'your tush and']


###  Entrenar modelo de predicción con diferentes matrices de palabras (variables predictoras)

In [19]:
# Definición de modelo Naive Bayes para predecir la varaible 'y' y variables predictoras x_A
nb = MultinomialNB()
pd.Series(cross_val_score(nb, X_dtm_A, y, cv=10)).describe()

,0
count,10.000000
mean,0.369444
std,0.158925
min,0.111111
25%,0.250000
50%,0.354167
75%,0.500000
max,0.625000


In [20]:
# Definición de modelo Naive Bayes para predecir la varaible 'y' y variables predictoras x_B
nb = MultinomialNB()
pd.Series(cross_val_score(nb, X_dtm_B, y, cv=10)).describe()

,0
count,10.000000
mean,0.368056
std,0.165278
min,0.111111
25%,0.281250
50%,0.375000
75%,0.486111
max,0.625000


In [21]:
# Definición de modelo Naive Bayes para predecir la varaible 'y' y variables predictoras x_B
nb = MultinomialNB()
pd.Series(cross_val_score(nb, X_dtm_C, y, cv=10)).describe()

,0
count,10.000000
mean,0.352778
std,0.118613
min,0.125000
25%,0.270833
50%,0.375000
75%,0.427083
max,0.500000


In [28]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import trigrams

# Descargar recursos necesarios
nltk.download('punkt')

texto = "Lo que todos tenemos que hacer es asegurarnos de que estamos usando la IA de una manera que sea en beneficio de la humanidad, no en detrimento de la humanidad"

# Tokenizar texto en palabras
tokens = word_tokenize(texto)

# Obtener trigramas
lista_trigramas = list(trigrams(tokens))

# Imprimir trigramas
print("Trigramas:")
for t in lista_trigramas:
    print(t)

# Imprimir cantidad de trigramas
print("\nNúmero de trigramas:", len(lista_trigramas))

Trigramas:
('Lo', 'que', 'todos')
('que', 'todos', 'tenemos')
('todos', 'tenemos', 'que')
('tenemos', 'que', 'hacer')
('que', 'hacer', 'es')
('hacer', 'es', 'asegurarnos')
('es', 'asegurarnos', 'de')
('asegurarnos', 'de', 'que')
('de', 'que', 'estamos')
('que', 'estamos', 'usando')
('estamos', 'usando', 'la')
('usando', 'la', 'IA')
('la', 'IA', 'de')
('IA', 'de', 'una')
('de', 'una', 'manera')
('una', 'manera', 'que')
('manera', 'que', 'sea')
('que', 'sea', 'en')
('sea', 'en', 'beneficio')
('en', 'beneficio', 'de')
('beneficio', 'de', 'la')
('de', 'la', 'humanidad')
('la', 'humanidad', ',')
('humanidad', ',', 'no')
(',', 'no', 'en')
('no', 'en', 'detrimento')
('en', 'detrimento', 'de')
('detrimento', 'de', 'la')
('de', 'la', 'humanidad')

Número de trigramas: 29


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
